# Get PC Information from Local Cure PCs
This script runs the following processes: Lookup local cure machine's pcid, ip address, and CPC software version

## Setup

### Import Libraries

In [ ]:
print("Initializing Python script...\n")

print("Importing libraries...\n")
# Standard Library
import os
import socket
import sys
from datetime import datetime
from pathlib import Path

import pandas as pd

# 3rd Party
from natsort import index_natsorted, order_by_index
from sqlalchemy import create_engine

### Settings

#### Set SQL Environment, Import Config File, Define Notebook Settings

In [ ]:
# Define environment
os.environ['ENV'] = 'production'

# Import global config file
base = Path().resolve().parents[2]
sys.path.insert(0, str(base / 'shared/global_config'))

# Import config file variables
import config

# Toggle max rows and max columns
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

#### Set CPC Variables

In [ ]:
source = r"C\Program Files\CPC Client"
client = "CPCClient.exe"

desired_version = "2023-03-13"

### Connect to Pyrometry Database on DEV/QUAL/PROD Server

In [ ]:
env_server = config.PROD_SERVER
env_database = config.PYRO_DATABASE

table_name = 'Equipment PCs'

print(f'Connecting to {env_server} server {env_database} database...\n')

db_conn_str = (
    f'mssql+pyodbc://{env_server}/{env_database}?'
    f'trusted_connection=yes&' 
    f'driver=ODBC+Driver+17+for+SQL+Server'
)

db_engine = create_engine(db_conn_str)

equip_pc_query = f'SELECT * FROM [{table_name}]'
equip_pcs = pd.read_sql(equip_pc_query, db_engine)
equip_pcs

## Functions

### Get IP Address with PCID

In [ ]:
def get_ip(pcid):
    try:
        return socket.gethostbyname(pcid)
    except:
        return None

### Get PCID with IP Address

In [ ]:
def get_pcid(ip_address):
    try:
        hostname = socket.gethostbyaddr(ip_address)[0]
        pcid = hostname.split('.')[0].upper()
        return pcid
    except:
        return None

### Get CPC Version with IP Address or PCID

In [ ]:
def get_cpc(ip, pcid, source, client):
    try:
        client_cpc = os.path.join('\\' + '\\' + str(ip), source, client)

        if not os.path.exists(client_cpc):
            client_cpc = os.path.join('\\' + '\\' + str(pcid), source, client)
        print(client_cpc)
        if os.path.exists(client_cpc):
            client_date = datetime.fromtimestamp(os.path.getmtime(client_cpc)).strftime('%Y-%m-%d')
            return str(client_date)
        return None
    except Exception:
        return None

## Run Code

### Extract Info from Local PCs

In [ ]:
equip_pcs['New_IP_Address'] = equip_pcs['PCID'].apply(lambda pcid: get_ip(pcid))
equip_pcs['IP_Address'] = equip_pcs.apply(lambda row: row['New_IP_Address'] if row['New_IP_Address'] is not None else row['IP_Address'], axis=1) # If New IP Address is None, previous IP Address is used

equip_pcs['New_PCID'] = equip_pcs['IP_Address'].apply(lambda ip: get_pcid(ip))
equip_pcs['PCID'] = equip_pcs.apply(lambda row: row['New_PCID'] if row['New_PCID'] is not None else row['PCID'], axis=1) # If New PCID is None, previous PCID is used

equip_pcs['New_CPC_Version'] = equip_pcs.apply(lambda row: get_cpc(row['IP_Address'], row['PCID'], source, client), axis=1)
equip_pcs['CPC_Version'] = equip_pcs.apply(lambda row: row['New_CPC_Version'] if row['New_CPC_Version'] is not None else row['CPC_Version'], axis=1) # If New CPC Version is None, previous CPC Version is used

equip_pcs = equip_pcs.drop(columns=['New_IP_Address', 'New_PCID', 'New_CPC_Version'])

equip_pcs['Database_Name'] = equip_pcs['Alt_Name'].apply(lambda alt_name: f"CPC_{alt_name.replace(' ', '')}")

indexer = index_natsorted(equip_pcs['PC'])
equip_pcs = equip_pcs.reindex(order_by_index(equip_pcs.index, indexer))
equip_pcs = equip_pcs.reset_index(drop=True)

equip_pcs


## Output to Database

### Equipment PCs Table

#### Production

In [ ]:
equip_pcs.to_sql(name=table_name, con=db_engine, if_exists='replace', index=False)

#### Development

In [ ]:
# print(f"Connecting to SQL Server database...\n")

# server_name = "UT02SQLDEVAGL"
# database_name = "Pyrometry"
# table_name = "Equipment PCs"

# conn_str = f'mssql+pyodbc://{server_name}/{database_name}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server'
# engine = create_engine(conn_str)

# equip_pcs.to_sql(name=table_name, con=engine, if_exists='replace', index=False)